In [9]:
#importing libraries
import speech_recognition as sr
import pyttsx3
import datetime
import os
import sys
import time
import json
import requests
import threading
from textblob import TextBlob
import matplotlib.pyplot as plt
import numpy as np

In [10]:
#Declaring Engine for voice and text 

text = "Hello, how can I help you ?"
engine = pyttsx3.init()
voices = engine.getProperty("voices")
engine.setProperty('voice', voices[1].id) #different voices. ID = 3 for german mostly. male, female...etc...
engine.setProperty('rate', 150)
engine.setProperty('volume', 0.9)

print(f"Bot Says : {text}")
engine.say(text)
engine.runAndWait()

Bot Says : Hello, how can I help you ?


In [11]:
# Now let declare the above as a function and re-use it
def speak_to_user(text):
    engine = pyttsx3.init()
    voices = engine.getProperty("voices")
    engine.setProperty('voice', voices[1].id) #different voices. ID = 3 for german mostly. male, female...etc...
    engine.setProperty('rate', 150)
    engine.setProperty('volume', 0.9)

    print(f"Bot Says : {text}")
    engine.say(text)
    engine.runAndWait()

speak_to_user("Hello Gautam, Nice to meet you !!")

Bot Says : Hello Gautam, Nice to meet you !!


In [12]:
# Intent recognition and NLP
#Intent recognition helps our bot understand what a user wants to do.
# Declare a class
#lets list out different intents
class IntentRecognizer:
    def __init__(self):
        self.intents = {
            'greeting': ['hello', 'hi', 'hey', 'good morning', 'good afternoon', 'good evening'],
            'goodbye': ['bye', 'goodbye', 'see you', 'exit', 'quit', 'stop'],
            'time': ['time', 'what time', 'current time', 'clock'],
            'date': ['date', 'today', 'what day', 'current date'],
            'weather': ['weather', 'temperature', 'forecast', 'rain', 'sunny'],
            'name': ['your name', 'who are you', 'what are you called'],
            'help': ['help', 'what can you do', 'commands', 'assist'],
            'joke': ['joke', 'funny', 'make me laugh', 'tell me something funny'],
            'calculate': ['calculate', 'math', 'plus', 'minus', 'multiply', 'divide', 'add', 'subtract']
        }

    def recognize_intent(self, text):

        # assignement bert base Intent classification model and integrate here 
        text = text.lower()
        # Now lets declare a loop for patterns
        for intent, patterns in self.intents.items():
            for pattern in patterns:
                if pattern in text:
                    return intent
                



    def extract_entities(self, text, intent):
        entities = {}

        if intent == "calculate":
            import re
            numbers = re.findall(r'\d+', text)
            entities['numbers'] = [int(n) for n in numbers]
            if 'plus' in text or 'add' in text or '+' in text:
                entities['operation'] = 'add'

            elif 'minus' in text or 'substract' in text or '-' in text:
                entities['operation'] = 'substract'

            elif 'multiply' in text or 'times' in text or '*' in text:
                entities['operation'] = 'multiply'
            
            elif 'divide' in text or 'devide by' in text or '/' in text:
                entities['operation'] = 'devide'

        


        if intent == "joke":
            pass
        return entities
    
intent_recogniser = IntentRecognizer()
test_phrases = [
    "Hello there",
    "what time is it?",
    "Tell me a joke",
    "calculate 5 plus 3",
    "Goodbye"
]


for phrase in test_phrases:
    intent = intent_recogniser.recognize_intent(phrase)
    entities = intent_recogniser.extract_entities(phrase, intent)
    print(f"Intent:- {intent}")
    print(f"Entities:- {entities}")
    
        




Intent:- greeting
Entities:- {}
Intent:- time
Entities:- {}
Intent:- joke
Entities:- {}
Intent:- calculate
Entities:- {'numbers': [5, 3], 'operation': 'add'}
Intent:- goodbye
Entities:- {}


In [13]:
#Lets now build our Voicebot
#We are using voice of "Alex"

class VoiceBot:
    def __init__(self, name='Alex'):
        self.name = name
        self.intent_recognizer = IntentRecognizer()
        self.is_active = True
        
        ## Initilize TTs engine 
        self.tts_engine = pyttsx3.init()
        self.tts_engine.setProperty('rate', 150)
        self.tts_engine.setProperty('volume', 0.9)

        #Initilize STT

        self.recognizer = sr.Recognizer()

    def speak(self, text):
        print(f" Bot To SAY{text}")
        self.tts_engine.say(text)
        self.tts_engine.runAndWait()

    def listen(self):
        # Here we will have to intialize microphone
        with sr.Microphone()as source:
            print("Listening....")
            self.recognizer.adjust_for_ambient_noise(source, duration=1)##adjust_for_ambient_noise() is a built in function for removing background noice
            try:
                audio = self.recognizer.listen(source, timeout=5, phrase_time_limit=5)
                text = self.recognozer.recognoze_google(audio)
                print(f"User :- {text}")
                return text.lower()
            

            except Exception as e:
                self.speak("Sorry I am having trouble understanding you")
                return None



    def process_intent(self, text):
        intent = self.intent_recognizer.recognize_intent(text)
        entities = self.intent_recognizer.extract_entities(text, intent)

        print(f"Detected Intent {intent}")

        if intent == "greeting":
            return f"Hello I'm {self.name}, your Voice assistant. How can I help you?"
        
        elif intent == 'goodbye':
            self.is_active = False
            return "Goodbye! Have a great day!"
        elif intent == 'time':
            current_time = datetime.datetime.now().strftime("%I:%M %p")
            return f"The current time is {current_time}"

        elif intent == 'date':
            current_date = datetime.datetime.now().strftime("%B %d, %Y")
            return f"Today is {current_date}"
        
        elif intent == 'name':
            return f"I'm {self.name}, your voice assistant created in Python!"

        elif intent == 'help':
            return "I can help you with time, date, simple calculations, jokes, and general conversation. Just speak to me!"
        
        elif intent == 'joke':
            jokes = [
                "Why don't scientists trust atoms? Because they make up everything!",
                "Why did the scarecrow win an award? He was outstanding in his field!",
                "Why don't eggs tell jokes? They'd crack each other up!",
                "What do you call a fake noodle? An impasta!"
            ]

            import random
            return random.choice(jokes)
        
        elif intent == 'weather':
            return "I'd love to help with weather, but I need a weather API key for that. For now, I suggest checking your local weather app!"
        
        
        
        elif intent == 'calculate':
            return self.handle_calculation(entities)
        
        else:
            return "I'm not sure how to help with that. Can you try asking something else?"
        
        


    def handle_calculation(self, entities):
        if 'numbers' not in entities or len(entities['numbers']) < 2:
            return "I need at least two numbers to calculate. Try saying something like 'calculate 5 plus 3'"
        
        numbers = entities['numbers']
        operation = entities.get('operation', 'add')

        if operation == 'add':
            result = sum(numbers)
            return f"The sum of {' plus '.join(map(str, numbers))} is {result}"
        
        elif operation == 'subtract':
            result = numbers[0] - numbers[1]
            return f"{numbers[0]} minus {numbers[1]} is {result}"
        
        elif operation == 'multiply':
            result = numbers[0] * numbers[1]
            return f"{numbers[0]} times {numbers[1]} is {result}"
        
        elif operation == 'divide':
            if numbers[1] == 0:
                return "I can't divide by zero!"
            result = numbers[0] / numbers[1]
            return f"{numbers[0]} divided by {numbers[1]} is {result}"
        
        return "I couldn't perform that calculation."
        


    def run(self):
        self.speak(f"Hello! I'm {self.name}, Say something to get started!")

        while self.is_active:
            user_input = self.listen()
            if user_input:
                response = self.process_intent(user_input)
                self.speak(response)
            time.sleep(0.5)

        print("Bot session time out")
    

In [ ]:
#Now lets try the above voicebot
bot = VoiceBot("Gauti")
bot.run()

 Bot To SAYHello! I'm Gauti, Say something to get started!
Listening....
 Bot To SAYSorry I am having trouble understanding you
Listening....
 Bot To SAYSorry I am having trouble understanding you
Listening....
 Bot To SAYSorry I am having trouble understanding you
Listening....
 Bot To SAYSorry I am having trouble understanding you
Listening....
 Bot To SAYSorry I am having trouble understanding you
Listening....
 Bot To SAYSorry I am having trouble understanding you


Exception ignored in: <function BSTR.__del__ at 0x00000254F4DB6A20>
Traceback (most recent call last):
  File "c:\Users\HP\miniconda3\Lib\site-packages\comtypes\_post_coinit\bstr.py", line 26, in __del__
    def __del__(self, _free: Callable[["BSTR"], Any] = _SysFreeString) -> None:
KeyboardInterrupt: 


Listening....
 Bot To SAYSorry I am having trouble understanding you
Listening....
 Bot To SAYSorry I am having trouble understanding you
Listening....
 Bot To SAYSorry I am having trouble understanding you
Listening....
 Bot To SAYSorry I am having trouble understanding you
Listening....
 Bot To SAYSorry I am having trouble understanding you
Listening....
 Bot To SAYSorry I am having trouble understanding you
Listening....
 Bot To SAYSorry I am having trouble understanding you
Listening....
 Bot To SAYSorry I am having trouble understanding you
Listening....
 Bot To SAYSorry I am having trouble understanding you
Listening....
 Bot To SAYSorry I am having trouble understanding you
Listening....
 Bot To SAYSorry I am having trouble understanding you
Listening....
 Bot To SAYSorry I am having trouble understanding you
Listening....
 Bot To SAYSorry I am having trouble understanding you
Listening....
 Bot To SAYSorry I am having trouble understanding you
Listening....
 Bot To SAYSorry I a

Exception ignored in: <function BSTR.__del__ at 0x00000254F4DB6A20>
Traceback (most recent call last):
  File "c:\Users\HP\miniconda3\Lib\site-packages\comtypes\_post_coinit\bstr.py", line 26, in __del__
    def __del__(self, _free: Callable[["BSTR"], Any] = _SysFreeString) -> None:
KeyboardInterrupt: 


Listening....
 Bot To SAYSorry I am having trouble understanding you
Listening....
 Bot To SAYSorry I am having trouble understanding you
Listening....
